# Monte Carlo Methods III: Parameter Estimation and Markov Chain

This hands-on is based on [Gravitational Wave Open Data Workshop 2024](https://github.com/gw-odw/odw-2024/).

## Introduction

Gravitational waves are ripples in spacetime predicted by Albert Einstein's General Theory of Relativity.
These waves propagate outward from sources such as merging black holes, neutron star collisions, or rapidly rotating neutron stars.
The first direct detection of gravitational waves, GW150914, was observed by the LIGO detectors in 2015, opening a new window into observing astrophysical phenomena previously inaccessible by electromagnetic observations alone.

Detecting and analyzing gravitational waves allows astronomers and physicists to study objects and phenomena that emit very little or no light, providing valuable insights into the behavior of gravity in extreme environments and the properties of dense objects like black holes and neutron stars.

In this hands-pon, we introduce Bayesian inference methods essential for estimating the parameters of gravitational wave signals detected by instruments such as LIGO and Virgo.
Specifically, we'll demonstrate parameter estimation using Monte Carlo Markov Chain.